In [ ]:
from transformers import GPT2Model, GPT2Tokenizer
import sys
import os
sys.path.append('/Users/sohom/Desktop/s_github/forge1')
import torch
from torch.optim import AdamW
from src.attention.optimizations.grouped_query_attention import GroupedQueryAttention, GQAConfig
from model_patcher import patch_attention_in_gpt2, add_rope_to_attention, remove_position_embeddings
from train_utils import  train_epoch, evaluate, prepare_wikitext2, LanguageModelingWrapper



: 

In [ ]:
# Load base model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = GPT2Model.from_pretrained('gpt2')
gpt2_config = base_model.config
# 1. Patch single attention variant
gqa_config = GQAConfig(
        num_query_heads=gpt2_config.n_head,  # 12 for GPT-2 small
        num_kv_heads=gpt2_config.n_head //3,  # Reduce KV heads by factor of 3
        head_dim=gpt2_config.n_embd // gpt2_config.n_head,  # 768/12 = 64
        dropout=gpt2_config.attn_pdrop
    )

print(gqa_config)

flash_model = patch_attention_in_gpt2(base_model, GroupedQueryAttention,gqa_config)


final_model = flash_model
final_model = LanguageModelingWrapper(final_model)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

tokenizer.pad_token = tokenizer.eos_token

if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

final_model.to(device)
# Prepare data
train_loader, eval_loader = prepare_wikitext2(tokenizer)

optimizers =  AdamW(final_model.parameters(), lr=5e-5)

for epoch in range (100):
    train_epoch(final_model, train_loader, optimizers, device)
    evaluate(final_model, eval_loader, device)

GQAConfig(num_query_heads=12, num_kv_heads=4, head_dim=64, dropout=0.1, max_seq_length=8192, use_bias=False)
Using random initialization for layer 0
Using random initialization for layer 1
Using random initialization for layer 2
Using random initialization for layer 3
Using random initialization for layer 4
Using random initialization for layer 5
Using random initialization for layer 6
Using random initialization for layer 7
Using random initialization for layer 8
Using random initialization for layer 9
Using random initialization for layer 10
Using random initialization for layer 11


Training:   0%|          | 1/2295 [00:39<25:02:00, 39.29s/it]

training loss :  7.822300434112549


Training:   0%|          | 2/2295 [01:12<22:49:14, 35.83s/it]

training loss :  7.3063788414001465


...

training loss :  5.922584056854248


Training:   1%|          | 18/2295 [11:11<23:28:52, 37.12s/it]

training loss :  5.935183048248291


Training:   1%|          | 19/2295 [11:48<23:20:10, 36.91s/it]

training loss :  5.938259124755859
